In [1]:
#default_exp train_slice

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [6]:
column='SeriesInstanceUID'

In [9]:
df_series = Meta.df_any.sort_values(['SeriesInstanceUID', "ImagePositionPatient2"])

In [12]:
df_series

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_4509f2560,CT,ID_4c16e232,ID_c174374b07,ID_002c9733b7,,-125.0,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0
ID_0969176c0,CT,ID_4c16e232,ID_c174374b07,ID_002c9733b7,,-125.0,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0
ID_2363aa3ef,CT,ID_4c16e232,ID_c174374b07,ID_002c9733b7,,-125.0,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0
ID_7b6119ddf,CT,ID_4c16e232,ID_c174374b07,ID_002c9733b7,,-125.0,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0
ID_a787384c2,CT,ID_4c16e232,ID_c174374b07,ID_002c9733b7,,-125.0,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_8eb7c45bc,CT,ID_984a3f15,ID_7891a70bf4,ID_fffde5ed33,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_cf66e9f08,CT,ID_984a3f15,ID_7891a70bf4,ID_fffde5ed33,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_6c779d850,CT,ID_984a3f15,ID_7891a70bf4,ID_fffde5ed33,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [20]:
sop = 'ID_0969176c0'
idx = df_series.index.get_loc(sop)

In [22]:
sid = df_series.loc[sop].SeriesInstanceUID

In [27]:
prev_row = df_series.iloc[idx-1]

In [28]:
prev_row.SeriesInstanceUID == sid

True

In [31]:
prev = self.fn(df_series.index[idx-1])

'ID_4509f2560'

In [ ]:
#export
class OpenCTs:
    def __init__(self, path): 
        self.fn = get_pil_fn(path)
        self.tt = ToTensor()
        
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [ ]:
#export
def pad_batch(x, pad_to=None, value=0):
    bs_pad = pad_to-x.shape[0]
    pad = [0]*len(x.shape)*2
    pad[-1] = bs_pad
    return F.pad(x, pad=pad, value=value)

In [ ]:
#export
class TfmSlice:
    def __init__(self,df,open_fn):
        self.open_fn = open_fn
        self.df = df
    
    def x(self, sid):
        sids = self.df.SOPInstanceUID[sid].values
        x = self.open_fn(sids)
        if self.pad_to is None: return x
        t = type(x)
        return t(pad_batch(x, pad_to=self.pad_to))
    
    def y(self, sid): 
        vals = self.df.loc[sid,htypes].values
        return TensorMultiCategory(vals).float()

In [ ]:
#export
class TfmSlice:
    def __init__(self, df, path): 
        self.fn = get_pil_fn(path)
        self.tt = ToTensor()
        self.df = df
    def __call__(self, item):
        idx = self.df.iloc[item]
        prev = self.df
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [ ]:
#export
def get_3d_dsrc(df, open_fn, grps=Meta.grps, cv_idx=0, splits=Meta.splits_any):
    df_series = df.sort_values(['SeriesInstanceUID', "ImagePositionPatient2"])
    tfm = TfmSOP(df_series, open_fn, pad_to)
    sids = df_series.index.unique()
    
    s1 = np.where(np.in1d(sids, group_cv(cv_idx,grps)))[0]
    s2 = np.where(np.in1d(sids, grps[cv_idx]))[0]
    dsrc = DataSource(sids, [[tfm.x],[fn2label,EncodedMultiCategorize(htypes)]], splits=splits)
    return dsrc